# Step 0

Imports

In [1]:
import torch
import torchvision
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
import os
import warnings
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from PIL import Image
import numpy as np

Device selection

In [2]:
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Hyperparameters

In [3]:
num_epochs = 2
batch_size = 24
lr = .000001
pathCircle   = '/home/arjun/Documents/DatasetCollection/shapes/circles'
pathTriangle = '/home/arjun/Documents/DatasetCollection/shapes/triangles'
pathSquare   = '/home/arjun/Documents/DatasetCollection/shapes/squares'

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

In [5]:
classes = ['circle', 'square', 'triangle']

Creating pandas dataframe

In [6]:
cir = [ 0. for i in range(100)]
tri = [ 2. for i in range(100)]
sq =  [ 1. for i in range(100)]
all = cir + tri + sq

In [7]:
circle_data = [os.path.join(pathCircle, f) for f in os.listdir(pathCircle) if f.endswith('.png')]
triangle_data = [os.path.join(pathTriangle, f) for f in os.listdir(pathTriangle) if f.endswith('.png')]
square_data = [os.path.join(pathSquare, f) for f in os.listdir(pathSquare) if f.endswith('.png')]

dfCir = pd.DataFrame({'Images':circle_data, 'Shape' : cir})
dfTri = pd.DataFrame({'Images':triangle_data, 'Shape' : tri})
dfSq = pd.DataFrame({'Images':square_data, 'Shape' : sq})

df = dfCir.append(dfTri, ignore_index=True)
df = df.append(dfSq, ignore_index=True)
df                                # Thus we have the complete dataframe

FileNotFoundError: [Errno 2] No such file or directory: '/home/arjun/Documents/DatasetCollection/shapes/circles'

Train-Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Images'], df['Shape'], test_size=0.2)


In [ ]:
print(X_train)

Creating custom DataSet

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        image_path = self.df.iloc[index, 0]
        label = self.df.iloc[index, 1]

        with Image.open(image_path) as img:
            image = np.array(img, dtype=np.uint8)
        image = image.astype(np.float32) / 255.0
        image = torch.from_numpy(image.transpose((2, 0, 1)))
        image = image.reshape(-1, 3, 28, 28)

        sample = tuple([image,torch.tensor(label)])
        # if self.transform:
        #     sample = self.transform(sample)
        return sample
    

In [ ]:
train_dataset = CustomDataset(pd.concat([X_train, y_train], axis=1),)#  , transform=transform)
test_dataset  = CustomDataset(pd.concat([X_test, y_test], axis=1)  )# , transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 1

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 12, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(192,120)
        self.fc2 = nn.Linear(120, 64)
        self.fc3 = nn.Linear(64, 3)

    def forward(self, x):
        # x -> (n, 3, 28, 28)
        # print(type(x))
        # print(x)
        # print(x.shape)
        out = self.pool(F.relu(self.conv1(x)))    # -> (6, 12, 12)
        out = self.pool(F.relu(self.conv2(out)))  # -> (12, 4, 4)
        out = out.view(-1, 12*4*4)                # -> (1, 192)
        out = F.relu(self.fc1(out))               # -> (1, 120)
        out = F.relu(self.fc2(out))               # -> (1, 64)
        out = self.fc3(out)                       # -> (1, 3)
        return out
    
model = CNN().to(device)

# Step 2

In [ ]:
lossCat = nn.CrossEntropyLoss().to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=lr)

# Step 3

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1} / {num_epochs} ")

    for image, label in  train_loader:
        image = image.to(device)
        label = label.to(device)
        # print(image.shape)
        image = image.reshape(-1,3,28,28)
        y_hat = model(image)
        label = torch.tensor(label, requires_grad=True)
        y_hat = torch.tensor(torch.argmax(y_hat, dim=1), dtype=torch.float64)
        # print(y_hat)
        loss = lossCat(label, y_hat)
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()

        # print(label)
    print(f" [LOSS = {loss:.3f}]\n")